# MiniProject02
 This is our implementation for COMP 551 miniproject 02


In [165]:
import numpy as np
import matplotlib.pyplot as plt
import os 
import re
from nltk.stem import WordNetLemmatizer 
from nltk.stem.porter import PorterStemmer
from sklearn.metrics import classification_report
import time

## Sorting the words by frequence

In [166]:
dir = 'C:/Users/Hassan/Documents/Semester 6/COMP 551/Projects/proj2_materials/train/'

lemmatizer = WordNetLemmatizer()
stemmaizer = PorterStemmer()

def preprocess_text(text):
    text = re.sub("<.*>", ' ', text)
    text = text.lower().replace('\n', ' ').split()
    for i in range(len(text)):
        text[i] = lemmatizer.lemmatize(text[i])
    return text



# build vocab
# build the vocabulary of all the words in the documents
# return words as a dictionary
def build_vocab():
    word_list = {}
    for review_dir in os.listdir(dir):
        for filename in os.listdir(dir + review_dir):
             with open(dir + review_dir + "/"  + filename, 'r',encoding="utf8") as f:
                    x= f.read()
                    text = preprocess_text(x)
                    for word in text:
                        if not (word in word_list):
                            word_list[word] = 1
                        else: 
                            word_list[word] += 1                     
    # sort words by increasing order
    sorted_words = sorted(word_list.items(), key = lambda kv : kv[1])
    return sorted_words


# sets the size of the feature vector
# splices the vocab dictionary
# returns the words as a list
def set_feature_size(sorted_words, size):
    vocab = sorted_words[len(sorted_words): len(sorted_words) - size - 1:-1]
    return vocab

## Build feature vector for instance


In [177]:
text_neg = [] # list of all negative reviews in training set
text_pos = [] # list of all positive reviews in training set


test_text = []

# load raw training and validation data into lists of strings
for filename in os.listdir(dir + '/neg'):
    with open(dir + '/neg/' +  filename, 'r',encoding="utf8") as f:
        text_neg.append(f.read())
        
for filename in os.listdir(dir + '/pos'):
    with open(dir + '/pos/' +  filename, 'r',encoding="utf8") as f:
        text_pos.append(f.read())
        
# training data
train_size = 10000        
text_train_neg = text_neg[:train_size]
text_train_pos = text_pos[:train_size]


# validation data
val_size = 2500
text_val_neg = text_neg[train_size:train_size + val_size]        
text_val_pos = text_pos[train_size:train_size + val_size]


In [178]:
# for a list of data points
# produce a matrix of features
def extract_data(text_list, vocab):
    data = np.zeros([len(text_list), len(vocab)]); 
    for i in range(len(data)):
        # process and tokenize text
        text = preprocess_text(text_list[i])
        for j in range(len(vocab)):
            if vocab[j][0] in text:
                data[i][j] = 1;
    return data

In [179]:
# build the vocab
vocab_size = 4000
vocab = build_vocab()
vocab = set_feature_size(vocab, vocab_size)

In [180]:
# training data
data_matrix_neg = extract_data(text_train_neg, vocab)
data_matrix_pos = extract_data(text_train_pos, vocab)

# validation data
data_matrix_val_neg = extract_data(text_val_neg, vocab)
data_matrix_val_pos = extract_data(text_val_pos, vocab)

## The Bernoulli Naive Bayes Classifier

In [181]:
 # the bernoulli naive bayes class construction
class BernoulliNB(object):

    def __init__(self, alpha=1.0):
        self.alpha = alpha

    def fit(self, X, y):
        count_sample = X.shape[0]
        separated = [[x for x, t in zip(X, y) if t == c] for c in np.unique(y)]
        self.class_log_prior_ = [np.log(len(i) / count_sample) for i in separated]
        count = np.array([np.array(i).sum(axis=0) for i in separated]) + self.alpha
        smoothing = 2 * self.alpha
        n_doc = np.array([len(i) + smoothing for i in separated])
        self.feature_prob_ = count / n_doc[np.newaxis].T
        return self

    def predict_log_proba(self, X):
        return [(np.log(self.feature_prob_) * x + \
                 np.log(1 - self.feature_prob_) * np.abs(x - 1)
                ).sum(axis=1) + self.class_log_prior_ for x in X]

    def predict(self, X):
        return np.argmax(self.predict_log_proba(X), axis=1)

## Prepare data for fitting and prediction

In [182]:
# Set up training data
# negative review
y_train_neg = np.array([0] * 10000)
x_train_neg = data_matrix_neg
# positive review
y_train_pos = np.array([1] * 10000)
x_train_pos = data_matrix_pos

x_train = np.append(x_train_neg, x_train_pos, axis = 0)
y_train = np.append(y_train_neg, y_train_pos)


# validation data
x_val_neg = data_matrix_val_neg
y_val_neg = np.array([0] * 2500)
x_val_pos = data_matrix_val_pos
y_val_pos = np.array([1] * 2500)

x_val = np.append(x_val_neg , x_val_pos, axis = 0)
y_val = np.append(y_val_neg, y_val_pos)

In [183]:
BNB = BernoulliNB() # create Bernoulli Naive Bayes Classifier


BNB.fit(x_train,y_train) # train/fit with training data


In [184]:
# predict on training set
start_time = time.time() 
prediction_train = BNB.predict(x_train)
# predict on validation set
prediction_val = BNB.predict(x_val)
end_time = time.time() # get time of training





In [185]:
# report performance on training set
report_train = classification_report(y_train, prediction_train)
print("training: " + report_train)

print('\n \n')

# report perfomance on validation set
report_val = classification_report(y_val , prediction_val)
print("validation" + report_val)

training:              precision    recall  f1-score   support

          0       0.85      0.83      0.84     10000
          1       0.83      0.85      0.84     10000

avg / total       0.84      0.84      0.84     20000


 

validation             precision    recall  f1-score   support

          0       0.81      0.81      0.81      2500
          1       0.81      0.81      0.81      2500

avg / total       0.81      0.81      0.81      5000



In [186]:
# Save report
with open('C:/Users/Hassan/Documents/Semester 6/COMP 551/Projects/proj2_materials/vocab_size_{}_train_size_{}.txt'.format(vocab_size, 2* train_size), 'w') as f:
    f.write("training runtime: {}s \n".format(end_time - start_time))
    
    f.write("dictionary size: {} \n".format(vocab_size))
    f.write('size of training set {} \n'.format( 2 * train_size))
    f.write('size of validation set {} \n'.format( 2 * val_size))
    f.write('=============PERFORMANCE ON TRAINING SET============= \n')
    f.write(report_train)
    f.write('\n \n \n \n')
    f.write('=============PERFORMANCE ON VALIDATION SET============= \n')
    f.write(report_val)